In [1]:
# Import libraries
import numpy as np
import pandas as pd
import holidays
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)




### Import and merge data to a single datafram

In [2]:
# Load the data
df_22 = pd.read_pickle('../../data_raw/df_filled_22.pkl')
df_23 = pd.read_pickle('../../data_raw/df_filled_23.pkl')


In [3]:
# Verify the data loaded correctly
print(df_22.head())
print(df_23.head())

timestamp           2022-02-01 00:00:00  2022-02-01 01:00:00  \
station_complex_id                                             
1                                  17.0                  5.0   
10                                100.0                 24.0   
100                                10.0                  3.0   
101                                26.0                 10.0   
118                                21.0                 11.0   

timestamp           2022-02-01 02:00:00  2022-02-01 03:00:00  \
station_complex_id                                             
1                                   6.0                 12.0   
10                                  7.0                  8.0   
100                                 0.0                  3.0   
101                                 9.0                  3.0   
118                                 9.0                  3.0   

timestamp           2022-02-01 04:00:00  2022-02-01 05:00:00  \
station_complex_id                    

In [4]:
# Merge the two dataframes
df = pd.concat([df_22, df_23], axis=0)


In [5]:
# View the first few rows of the dataframe
df.sample(5)


timestamp,2022-02-01 00:00:00,2022-02-01 01:00:00,2022-02-01 02:00:00,2022-02-01 03:00:00,2022-02-01 04:00:00,2022-02-01 05:00:00,2022-02-01 06:00:00,2022-02-01 07:00:00,2022-02-01 08:00:00,2022-02-01 09:00:00,...,2023-12-31 14:00:00,2023-12-31 15:00:00,2023-12-31 16:00:00,2023-12-31 17:00:00,2023-12-31 18:00:00,2023-12-31 19:00:00,2023-12-31 20:00:00,2023-12-31 21:00:00,2023-12-31 22:00:00,2023-12-31 23:00:00
station_complex_id,,,,,,,,,,,,,,,,,,,,,
262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,158.0,154.0,140.0,124.0,114.0,144.0,119.0,102.0,66.0,28.0
306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,219.0,219.0,194.0,207.0,193.0,173.0,190.0,209.0,162.0,111.0
432,9.0,0.0,6.0,15.0,38.0,142.0,239.0,326.0,291.0,205.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,26.0,10.0,13.0,32.0,180.0,694.0,1015.0,1197.0,1059.0,781.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.0,63.0,29.0,31.0,27.0,41.0,23.0,23.0,20.0,12.0


In [6]:
# Reset the index
df.reset_index(inplace=True)


In [7]:
# Convert to long format
df_long = df.melt(id_vars='station_complex_id', var_name='timestamp', value_name='ridership')


In [8]:
# View the first few rows of the dataframe and shape
display(df_long.sample(5))
df_long.shape

,station_complex_id,timestamp,ridership
4066579,461,2022-09-23 16:00:00,1139.0
167803,601,2022-02-10 16:00:00,2443.0
11210484,9,2023-12-10 22:00:00,727.0
10592746,49,2023-11-05 07:00:00,NaN
4714494,372,2022-10-31 01:00:00,NaN


(11575104, 3)

In [9]:
# Convert the timestamp to datetime
df_long['timestamp'] = pd.to_datetime(df_long['timestamp'])


In [10]:
# Sort the dataframe
df_long.sort_values(['station_complex_id', 'timestamp'], inplace=True)


In [11]:
# Reset the index
df_long.reset_index(drop=True, inplace=True)


In [12]:
# View the first few rows of the dataframe
display(df_long.head(10))


,station_complex_id,timestamp,ridership
0,1,2022-02-01 00:00:00,17.0
1,1,2022-02-01 00:00:00,NaN
2,1,2022-02-01 01:00:00,5.0
3,1,2022-02-01 01:00:00,NaN
4,1,2022-02-01 02:00:00,6.0
5,1,2022-02-01 02:00:00,NaN
6,1,2022-02-01 03:00:00,12.0
7,1,2022-02-01 03:00:00,NaN
8,1,2022-02-01 04:00:00,44.0
9,1,2022-02-01 04:00:00,NaN


### Remove duplicates in the dataframe

In [13]:
# Sort the DataFrame so that the rows without NaN in ridership are first
df_long.sort_values(by=['station_complex_id', 'timestamp', 'ridership'], ascending=[True, True, False], inplace=True)

# Drop duplicates based on station_complex_id and timestamp, keeping the first non-NaN ridership
df_long.drop_duplicates(subset=['station_complex_id', 'timestamp'], keep='first', inplace=True)

# Display the updated DataFrame
print("Data shape after dropping NaNs in duplicates:", df_long.shape)
display(df_long.head(10))


Data shape after dropping NaNs in duplicates: (5787552, 3)


,station_complex_id,timestamp,ridership
0,1,2022-02-01 00:00:00,17.0
2,1,2022-02-01 01:00:00,5.0
4,1,2022-02-01 02:00:00,6.0
6,1,2022-02-01 03:00:00,12.0
8,1,2022-02-01 04:00:00,44.0
10,1,2022-02-01 05:00:00,143.0
12,1,2022-02-01 06:00:00,405.0
14,1,2022-02-01 07:00:00,735.0
16,1,2022-02-01 08:00:00,950.0
18,1,2022-02-01 09:00:00,512.0


In [14]:
# view the info of the dataframe
df_long.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5787552 entries, 0 to 11575103
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   station_complex_id  object        
 1   timestamp           datetime64[ns]
 2   ridership           float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 176.6+ MB


In [15]:
# Check for missing values
print(df_long.isnull().sum())
df_long.head(10)


station_complex_id    0
timestamp             0
ridership             0
dtype: int64


,station_complex_id,timestamp,ridership
0,1,2022-02-01 00:00:00,17.0
2,1,2022-02-01 01:00:00,5.0
4,1,2022-02-01 02:00:00,6.0
6,1,2022-02-01 03:00:00,12.0
8,1,2022-02-01 04:00:00,44.0
10,1,2022-02-01 05:00:00,143.0
12,1,2022-02-01 06:00:00,405.0
14,1,2022-02-01 07:00:00,735.0
16,1,2022-02-01 08:00:00,950.0
18,1,2022-02-01 09:00:00,512.0


### Extract date info to columns

In [16]:
# Extract the hour, day of the week, month, and weekend
df_long['hour'] = df_long['timestamp'].dt.hour
df_long['day_of_week'] = df_long['timestamp'].dt.dayofweek  # Monday=0, Sunday=6
df_long['month'] = df_long['timestamp'].dt.month
df_long['is_weekend'] = df_long['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)


In [17]:
df_long.sample(5)

,station_complex_id,timestamp,ridership,hour,day_of_week,month,is_weekend
10662277,636,2023-02-21 18:00:00,1982.0,18,1,2,0
8075347,436,2023-09-23 09:00:00,95.0,9,5,9,1
3150144,262,2022-07-15 00:00:00,14.0,0,4,7,0
11229132,89,2022-06-21 06:00:00,149.0,6,1,6,0
5522068,35,2022-06-28 02:00:00,15.0,2,1,6,0


### Include holidays

In [18]:
# Use Holidays library to get the US holidays
us_holidays = holidays.US()

In [19]:
# Create a new column for holidays
df_long['holiday'] = df_long['timestamp'].apply(lambda x: x in us_holidays).astype(int)


In [20]:
# Add holiday names
df_long['holiday_name'] = df_long['timestamp'].apply(lambda x: us_holidays.get(x)).fillna('No Holiday')

In [21]:
display(df_long.sample(5))
df_long.info()

,station_complex_id,timestamp,ridership,hour,day_of_week,month,is_weekend,holiday,holiday_name
7878979,43,2023-07-02 09:00:00,166.0,9,6,7,1,0,No Holiday
868164,150,2022-03-23 18:00:00,64.0,18,2,3,0,0,No Holiday
1811643,186,2023-02-01 13:00:00,63.0,13,2,2,0,0,No Holiday
1062652,156,2022-05-06 14:00:00,182.0,14,4,5,0,0,No Holiday
11226638,89,2022-04-30 07:00:00,0.0,7,5,4,1,0,No Holiday


<class 'pandas.core.frame.DataFrame'>
Index: 5787552 entries, 0 to 11575103
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   station_complex_id  object        
 1   timestamp           datetime64[ns]
 2   ridership           float64       
 3   hour                int32         
 4   day_of_week         int32         
 5   month               int32         
 6   is_weekend          int64         
 7   holiday             int64         
 8   holiday_name        object        
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(2)
memory usage: 375.3+ MB


In [22]:
# Set the timestamp as the index
df_long.set_index('timestamp', inplace=True)

In [23]:
# Make a copy of the dataframe to use for the next steps
df_long_copy_1 = df_long.copy()

### Create lag features for modeling

In [24]:
# Create lag features
df_long_copy_1['ridership_lag_1'] = df_long_copy_1.groupby('station_complex_id')['ridership'].shift(1)
df_long_copy_1['ridership_lag_3'] = df_long_copy_1.groupby('station_complex_id')['ridership'].shift(3)
df_long_copy_1['ridership_lag_7'] = df_long_copy_1.groupby('station_complex_id')['ridership'].shift(7)
df_long_copy_1['ridership_lag_24'] = df_long_copy_1.groupby('station_complex_id')['ridership'].shift(24)
df_long_copy_1['rolling_mean_6h'] = df_long_copy_1.groupby('station_complex_id')['ridership'].transform(lambda x: x.rolling(window=6).mean())
df_long_copy_1['rolling_std_6h'] = df_long_copy_1.groupby('station_complex_id')['ridership'].transform(lambda x: x.rolling(window=6).std())

In [25]:
# View the first few rows of the dataframe
display(df_long_copy_1.head(10))

,station_complex_id,ridership,hour,day_of_week,month,is_weekend,holiday,holiday_name,ridership_lag_1,ridership_lag_3,ridership_lag_7,ridership_lag_24,rolling_mean_6h,rolling_std_6h
timestamp,,,,,,,,,,,,,,
2022-02-01 00:00:00,1,17.0,0,1,2,0,0,No Holiday,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-01 01:00:00,1,5.0,1,1,2,0,0,No Holiday,17.0,NaN,NaN,NaN,NaN,NaN
2022-02-01 02:00:00,1,6.0,2,1,2,0,0,No Holiday,5.0,NaN,NaN,NaN,NaN,NaN
2022-02-01 03:00:00,1,12.0,3,1,2,0,0,No Holiday,6.0,17.0,NaN,NaN,NaN,NaN
2022-02-01 04:00:00,1,44.0,4,1,2,0,0,No Holiday,12.0,5.0,NaN,NaN,NaN,NaN
2022-02-01 05:00:00,1,143.0,5,1,2,0,0,No Holiday,44.0,6.0,NaN,NaN,37.833333,53.461824
2022-02-01 06:00:00,1,405.0,6,1,2,0,0,No Holiday,143.0,12.0,NaN,NaN,102.500000,157.211641
2022-02-01 07:00:00,1,735.0,7,1,2,0,0,No Holiday,405.0,44.0,17.0,NaN,224.166667,291.654190
2022-02-01 08:00:00,1,950.0,8,1,2,0,0,No Holiday,735.0,143.0,5.0,NaN,381.500000,388.851000


In [26]:
# Apply backfill to fill NaN values where possible
df_long_copy_1.bfill(inplace=True)

# Apply forward fill again to ensure no NaN values remain
df_long_copy_1.ffill(inplace=True)

# Check DF for missing values
print("Data shape after backfill and forward filling NaNs:", df_long_copy_1.shape)
display(df_long_copy_1.head(10))


Data shape after backfill and forward filling NaNs: (5787552, 14)


,station_complex_id,ridership,hour,day_of_week,month,is_weekend,holiday,holiday_name,ridership_lag_1,ridership_lag_3,ridership_lag_7,ridership_lag_24,rolling_mean_6h,rolling_std_6h
timestamp,,,,,,,,,,,,,,
2022-02-01 00:00:00,1,17.0,0,1,2,0,0,No Holiday,17.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 01:00:00,1,5.0,1,1,2,0,0,No Holiday,17.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 02:00:00,1,6.0,2,1,2,0,0,No Holiday,5.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 03:00:00,1,12.0,3,1,2,0,0,No Holiday,6.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 04:00:00,1,44.0,4,1,2,0,0,No Holiday,12.0,5.0,17.0,17.0,37.833333,53.461824
2022-02-01 05:00:00,1,143.0,5,1,2,0,0,No Holiday,44.0,6.0,17.0,17.0,37.833333,53.461824
2022-02-01 06:00:00,1,405.0,6,1,2,0,0,No Holiday,143.0,12.0,17.0,17.0,102.500000,157.211641
2022-02-01 07:00:00,1,735.0,7,1,2,0,0,No Holiday,405.0,44.0,17.0,17.0,224.166667,291.654190
2022-02-01 08:00:00,1,950.0,8,1,2,0,0,No Holiday,735.0,143.0,5.0,17.0,381.500000,388.851000


In [27]:
# Count of null values
df_long_copy_1.isnull().sum()

station_complex_id    0
ridership             0
hour                  0
day_of_week           0
month                 0
is_weekend            0
holiday               0
holiday_name          0
ridership_lag_1       0
ridership_lag_3       0
ridership_lag_7       0
ridership_lag_24      0
rolling_mean_6h       0
rolling_std_6h        0
dtype: int64

In [28]:
# Create a copy of the dataframe
df_long_copy_2 = df_long_copy_1.copy()
df_long_copy_2.head(5)

,station_complex_id,ridership,hour,day_of_week,month,is_weekend,holiday,holiday_name,ridership_lag_1,ridership_lag_3,ridership_lag_7,ridership_lag_24,rolling_mean_6h,rolling_std_6h
timestamp,,,,,,,,,,,,,,
2022-02-01 00:00:00,1,17.0,0,1,2,0,0,No Holiday,17.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 01:00:00,1,5.0,1,1,2,0,0,No Holiday,17.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 02:00:00,1,6.0,2,1,2,0,0,No Holiday,5.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 03:00:00,1,12.0,3,1,2,0,0,No Holiday,6.0,17.0,17.0,17.0,37.833333,53.461824
2022-02-01 04:00:00,1,44.0,4,1,2,0,0,No Holiday,12.0,5.0,17.0,17.0,37.833333,53.461824


In [ ]:
# Export to csv
df_long_copy_2.to_csv('../../Modeling_preprocess_clean/cleaned_data/ridership_w_lag.csv', index=True)
